On branch rakeshw_final
Your branch is up to date with 'origin/rakeshw_final'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   EDA/Augment_Missing_Data.ipynb
	modified:   EDA/Data_Clean.ipynb
	modified:   EDA/EDA_Final.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	cleantrain/
	cleantrain_8/
	data/final_check_sum.p
	data/id_lookup.p
	data/test.csv
	data/test.p
	data/testdup.p
	data/train.p
	data/traindup.p
	data/training.csv
	models/Rakesh_models.ipynb
	utils/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
#Set the utils path to point to the utils directory locally
UTILS_PATH = "utils/"

import os, sys
sys.path.append(UTILS_PATH)
from load_models import LoadTrainModels
from predict_models import PredictModels
from transform_data import TransformData
import imp
import pickle
import numpy as np
import pandas as pd
from keras.models import Sequential, Model, model_from_json

In [ ]:
### Must be in ../blackboxes
### this is gold right here.
file_path = "cleantrain/"
trainer = LoadTrainModels("data/models/", file_path)

#Adding a transform data object if I want to do any transformations in this file
transformdata = TransformData()
trainer.print_paths()

Model dir: data/models/
